In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time, json
import sys
sys.path.append("../")
sys.path.append("../chess_llm_interpretability")
import os
import torch
import numpy as np

import logging
import src.utils.logging_utils as logging_utils
import src.utils.env_utils as env_utils
import src.functional as functional
import src.models as models
import src.tokens as tokens

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

2024-10-24 02:46:28 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_KEY = "google/gemma-2-9b-it"

# MODEL_KEY = "google/gemma-2-2b"
# MODEL_KEY = "meta-llama/Llama-3.2-3B"
# MODEL_KEY = "meta-llama/Llama-3.1-8B"

mt = models.ModelandTokenizer(
    model_key=MODEL_KEY,
    torch_dtype=torch.bfloat16,
)

2024-10-24 02:46:28 src.models WARNING  meta-llama/Llama-3.2-3B-Instruct not found in models
If not found in cache, model will be downloaded from HuggingFace to cache directory


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

2024-10-24 02:49:06 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

2024-10-24 02:49:09 src.models INFO     loaded model <meta-llama/Llama-3.2-3B-Instruct> | size: 6127.841 MB | dtype: torch.bfloat16 | device: cuda:0


In [4]:
true_prompt = "The city of Paris is in the country of France."
false_prompt = "The city of Paris is in the country of Italy."
true_input, false_input = [tokens.prepare_input(p, mt) for p in (true_prompt, false_prompt)]

In [5]:
mt.n_layer

28

In [7]:
true_h, false_h = [
    functional.get_hs(
        mt = mt,
        input = input_,
        locations = [(mt.layer_name_format.format(24), input_.input_ids.shape[1] - 1)]
    ) for input_ in (true_input, false_input)
]

true_h.shape

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([3072])

In [8]:
true_token, false_token = [mt.tokenizer.encode(t)[-1] for t in ("True", "False")]
[mt.tokenizer.decode(t) for t in (true_token, false_token)]

['True', 'False']

In [9]:
base_prompt = '''True or false: placeholder placeholder placeholder placeholder placeholder

'''
instruct_prompt = '''<|start_header_id|>user<|end_header_id|>

True or false: placeholder placeholder placeholder placeholder placeholder<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

'''
target_prompt = {
    "meta-llama/Llama-3.2-3B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.1-8B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.2-3B" : base_prompt,
}[MODEL_KEY]

for prompt, h in zip((true_prompt, false_prompt), (true_h, false_h)):
    print(prompt)
    _, result_dict = functional.patchscope(
        mt = mt, 
        h = h,
        target_prompt = target_prompt,
        layer_idx = 3,
        interested_tokens = (true_token, false_token),
        k = 5)
    for t in (true_token, false_token):
        print("   ", result_dict[t])


The city of Paris is in the country of France.
    (15, PredictedToken(token='True', prob=0.0007195635116659105, logit=16.625, token_id=2575))
    (13, PredictedToken(token='False', prob=0.0011863596737384796, logit=17.125, token_id=4139))
The city of Paris is in the country of Italy.
    (15, PredictedToken(token='True', prob=0.00038651248905807734, logit=17.0, token_id=2575))
    (12, PredictedToken(token='False', prob=0.001528686610981822, logit=18.375, token_id=4139))
